### **Búsqueda y Minería de Información 2022-23**
### Universidad Autónoma de Madrid, Escuela Politécnica Superior
### Grado en Ingeniería Informática, 4º curso
# **Implementación de un motor de búsqueda**

Fechas:

* Comienzo: martes 7 / jueves 9 de febrero
* Entrega: martes 21 / jueves 23 de febrero (14:00)

# Introducción

## Autores

Guillermo Martín-Coello Juarez & Daniel Varela Sanchez

## Objetivos

Los objetivos de esta práctica son:

* La iniciación a la implementación de un motor de búsqueda.
*	Una primera comprensión de los elementos básicos necesarios para implementar un motor completo.
*	La iniciación al uso de la librería [Whoosh](https://whoosh.readthedocs.io/en/latest/intro.html) en Python para la creación y utilización de índices, funcionalidades de búsqueda en texto.
*	La iniciación a la implementación de una función de ránking sencilla.

Los documentos que se indexarán en esta práctica, y sobre los que se realizarán consultas de búsqueda serán documentos HTML, que deberán ser tratados para extraer y procesar el texto contenido en ellos. 

La práctica plantea como punto de partida una pequeña API general sencilla (y cuyo uso se puede ver en un programa de prueba que se encuentra al final del enunciado), que pueda implementarse de diferentes maneras, como así se hará en esta práctica y las siguientes. A modo de toma de contacto y arranque de la asignatura, en esta primera práctica se completará una implementación de la API utilizando Whoosh, con lo que resultará bastante trivial la solución (en cuanto a la cantidad de código a escribir). En la siguiente práctica el estudiante desarrollará sus propias implementaciones, sustituyendo el uso de Whoosh que vamos a hacer en esta primera práctica.

En términos de operaciones propias de un motor de búsqueda, en esta práctica el estudiante se encargará fundamentalmente de:

a) En el proceso de indexación: recorrer los documentos de texto de una colección dada, eliminar del contenido posibles marcas tales como html, y enviar el texto a indexar por parte de Whoosh. 

b) En el proceso de responder consultas: implementar una primera versión sencilla de una o dos funciones de ránking en el modelo vectorial, junto con alguna pequeña estructura auxiliar.

## Material proporcionado

Se proporcionan (bien en el curso de Moodle o dentro de este documento):

*	Varias clases e interfaces Python (mayormente incompletas) a lo largo de este *notebook*, desde las que el estudiante partirá para completar código e integrará con ellas las suyas propias. 
La celda de prueba *al final de este notebook* implementa un programa que deberá funcionar con el código a implementar por el estudiante. Además, se proporciona a continuación una celda con código ejemplo que ilustra las funciones más útiles de la API de Whoosh.
*	Una pequeña colección <ins>docs1k.zip</ins> con aproximadamente 1.000 documentos HTML, y un pequeño fichero <ins>urls.txt</ins>. Ambas representan colecciones de prueba para depurar las implementaciones y comprobar su corrección.
*	Un documento de texto <ins>output.txt</ins> con la salida estándar que deberá producir la ejecución de la celda de prueba (salvo los tiempos de ejecución que pueden cambiar, aunque la tendencia en cuanto a qué métodos tardan más o menos debería cumplirse).

## Ejemplo API Whoosh

En la siguiente celda de código se incluyen varios ejemplos para comprobar cómo usar la API de la librería *Whoosh*.

In [10]:
# Whoosh API
import whoosh
from whoosh.fields import Schema, TEXT, ID
from whoosh.formats import Format
from whoosh.qparser import QueryParser
from urllib.request import urlopen
from bs4 import BeautifulSoup
import os, os.path
import shutil

Document = Schema(
        path=ID(stored=True),
        content=TEXT(vector=Format))

def whooshexample_buildindex(dir, urls):
    if os.path.exists(dir): shutil.rmtree(dir)
    os.makedirs(dir)
    writer = whoosh.index.create_in(dir, Document).writer()
    for url in urls:
        writer.add_document(path=url, content=BeautifulSoup(urlopen(url).read(), "lxml").text)
    writer.commit()

def whooshexample_search(dir, query):
    index = whoosh.index.open_dir(dir)
    searcher = index.searcher()
    qparser = QueryParser("content", schema=index.schema)
    print("Search results for '", query, "'")
    for docid, score in searcher.search(qparser.parse(query)).items():
        print(score, "\t", index.reader().stored_fields(docid)['path'])
    print()

def whooshexample_examine(dir, term, docid, n):
    reader = whoosh.index.open_dir(dir).reader()
    print("Total nr. of documents in the collection:", reader.doc_count())
    print("Total frequency of '", term, "':", reader.frequency("content", term))
    print("Nr. documents containing '", term, "':", reader.doc_frequency("content", term))
    for p in reader.postings("content", term).items_as("frequency") if reader.doc_frequency("content", term) > 0 else []:
        print("\tFrequency of '", term, "' in document", p[0], ":", p[1])
    raw_vec = reader.vector(docid, "content")
    raw_vec.skip_to(term)
    if raw_vec.id() == term:
        print("Frequency of '", raw_vec.id(), "' in document", docid, reader.stored_fields(docid)['path'], ":", raw_vec.value_as("frequency"))
    else:
        print("Term '", term, "' not found in document", docid)
    print("Top", n, "most frequent terms in document", docid, reader.stored_fields(docid)['path']) 
    vec = reader.vector(docid, "content").items_as("frequency")
    for p in sorted(vec, key=lambda x: x[1], reverse=True)[0:n]:
        print("\t", p)
    print()

urls = ["https://en.wikipedia.org/wiki/Simpson's_paradox", 
        "https://en.wikipedia.org/wiki/Bias",
        "https://en.wikipedia.org/wiki/Entropy"]

dir = "index/whoosh/example/urls"

whooshexample_buildindex(dir, urls)
whooshexample_search(dir, "probability")
whooshexample_examine(dir, "probability", 0, 5)

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)>

## Calificación

Esta práctica se calificará con una puntuación de 0 a 10 atendiendo a las puntuaciones individuales de ejercicios y apartados dadas en el enunciado.  

El peso de la nota de esta práctica en la calificación final de prácticas es del **20%**.

La calificación se basará en a) el **número** de ejercicios realizados y b) la **calidad** de los mismos. 
La puntuación que se indica en cada apartado es orientativa, en principio se aplicará tal cual se refleja pero podrá matizarse por criterios de buen sentido si se da el caso.

Para dar por válida la realización de un ejercicio, el código deberá funcionar (a la primera) **sin ninguna modificación**. El profesor comprobará este aspecto ejecutando la celda de prueba así como otras pruebas adicionales.

## Entrega

La entrega consistirá en un único fichero tipo *notebook* donde se incluirán todas las **implementaciones** solicitadas en cada ejercicio, así como una explicación de cada uno a modo de **memoria**. Si se necesita entregar algún fichero adicional (por ejemplo, imágenes) se puede subir un fichero ZIP a la tarea correspondiente de Moodle. En cualquiera de los dos casos, el nombre del fichero a subir será **bmi-p1-XX**, donde XX debe sustituirse por el número de pareja (01, 02, ..., 10, ...).

En concreto, se debe documentar:

- Qué version(es) del modelo vectorial se ha(n) implementado en el ejercicio 2.
- Cómo se ha conseguido colocar un documento en la primera posición de ránking, para cada buscador implementado en el ejercicio 2.
- El trabajo realizado en el ejercicio 3. 
- Y cualquier otro aspecto que el estudiante considere oportuno destacar.


## Indicaciones

Se podrán definir clases adicionales a las que se indican en el enunciado, por ejemplo, para reutilizar código. Y el estudiante podrá utilizar o no el software que se le proporciona, con la siguiente limitación: 

*	No deberá editarse el código proporcionado más allá de donde se indica explícitamente.
*	**La celda de prueba deberá ejecutar** correctamente sin ninguna modificación.

# Ejercicio 1: Implementación basada en Whoosh

Implementar las clases y módulos necesarios para que la celda de prueba funcione. Se deja al estudiante deducir alguna de las relaciones jerárquicas entre las clases Python.

## Ejercicio 1.1: Indexación (3.5pt)

Definir las siguientes clases:

* Index: clase general (no depende de Whoosh) y que encapsule los métodos necesarios para que funcione la celda de prueba que se encuentra al final del enunciado.
* Builder: clase general (no depende de Whoosh) que permite construir un índice (a través del método Builder.build()), tal y como se llama desde la celda de prueba entregada.
* WhooshIndex: clase que cumpla con la interfaz definida en *Index* usando la librería de whoosh.
* WhooshBuilder: clase que cumpla con la interfaz definida en *Builder* pero que use internamente la librería de whoosh.

La entrada para construir el índice (método Builder.build()) podrá ser, tal y como se puede ver en el programa de prueba al final de este notebook, a) un fichero de texto con direcciones Web (una por línea); b) una carpeta del disco (se indexarán todos los ficheros de la carpeta, sin entrar en subcarpetas); o c) un archivo zip que contiene archivos comprimidos a indexar. Para simplificar, supondremos que el contenido a indexar es siempre HTML.

In [1]:
import whoosh
from whoosh.fields import Schema, TEXT, ID
from whoosh.formats import Format
from whoosh.qparser import QueryParser
from zipfile import ZipFile
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl


# A schema in Whoosh is the set of possible fields in a document in the search space. 
# We just define a simple 'Document' schema, with a path (a URL or local pathname)
# and a content.
Document = Schema(
        path=ID(stored=True),
        content=TEXT(vector=Format))

class WhooshBuilder():
    """
    Whoosh Builder:
    Class to build a Whoosh index from a collection of documents.
    The collection can be a directory of files, a zip file, or a text file with a list of URLs.

    @param index_path : str
        Path to the index directory.
    @attribute index_path : str
        Path to the index directory.
    @attribute writer : whoosh.index.Writer
        Whoosh index writer.
    """
    index_path = None
    writer = None
    
    def __init__(self, index_path):
        self.index_path = index_path
        

    def build(self, collections):
        """
        build(collections)
            - Build the index from a collection of documents.
            - The collection can be a directory of files, a zip file, or a text file with a list of URLs.
        @param collections: path to the collection.
        """
        ctx = ssl.create_default_context()
        ctx.check_hostname = False
        ctx.verify_mode = ssl.CERT_NONE
        if os.path.exists(self.index_path): shutil.rmtree(self.index_path)
        os.makedirs(self.index_path)

        self.writer = whoosh.index.create_in(self.index_path, Document).writer()
        if os.path.isdir(collections):
            for path in sorted(os.listdir(collections)):
                fn=os.path.join(collections, path)
                if os.path.isfile(fn):
                    with open(fn, 'r') as f:
                        self.writer.add_document(path=fn, content=f.read())

        elif collections.endswith(".txt"):
            with open(collections) as f:
                    content = f.readlines()
                    for l in content:
                        l = l.strip()
                        self.writer.add_document(path=l, content=BeautifulSoup(urlopen(l, context=ctx).read(), "html.parser").text)

        elif collections.endswith(".zip"):
            for l in ZipFile(collections, "r").namelist():
                    self.writer.add_document(path=l, content=BeautifulSoup(ZipFile(collections, "r").read(l), "html.parser").text)
        return

    def commit(self):
        """
        commit()
            - Commit the index.
        """
        self.writer.commit()

class WhooshIndex():
    """
    Whoosh Index:
    Class to access a Whoosh index.
    
    @param dir : str
        Path to the index directory.
    @attribute index : whoosh.index.Index
        Whoosh index.
    @attribute reader : whoosh.index.Reader
        Whoosh index reader.
    @attribute path : str
        Path to the index directory.
    """
    reader = None
    index = None
    path = ""

    def __init__(self, dir) -> None:
        self.index = whoosh.index.open_dir(dir)
        self.reader = self.index.reader()
        self.path=dir

    def ndocs(self):
        """
        ndocs()
            - Return the number of documents in the index.
        """
        return self.reader.doc_count()

    def all_terms(self):
        """
        all_terms()
            - Return a list of all terms in the index.
        """
        return list(self.reader.field_terms("content"))

    def all_terms_with_freq(self):
        """
        all_terms_with_freq()
            - Return a list of all terms in the index with their frequency.
        """
        res=[]
        for i in self.all_terms():
            res.append((i, self.total_freq(i)))
        return res

    def total_freq(self, term):
        """
        total_freq(term)
            - Return the total frequency of a term in the index.
        @param term: term to search
        """
        return self.reader.frequency("content", term)

    def doc_path(self, doc_id):
        """
        doc_path(doc_id)
            - Return the path of a document given its id.
            @param doc_id: id of the document
        """
        return self.reader.stored_fields(doc_id)['path']

    def term_freq(self, term, doc_id):
        """
        term_freq(term, doc_id)
            - Return the frequency of a term in a document given its id.
            @param term: term to search
            @param doc_id: id of the document
        """
        for p in self.reader.postings("content", term).items_as("frequency") if self.reader.doc_frequency("content", term) > 0 else []:
            if doc_id == p[0]:
                return p[1]
        return 0

    def doc_freq(self, term):
        """
        doc_freq(term)
            - Return the document frequency of a term in the index.
            @param term: term to search
        """
        return self.reader.doc_frequency("content", term)

    def doc_terms_with_freq(self, doc_id):
        """
        doc_terms_with_freq(doc_id)
            - Return a list of terms in a document given its id.
            @param doc_id: id of the document
        """
        freqs = []
        for term in self.reader.vector_as("frequency", doc_id, "content"):
            freqs.append((term[0], term[1]))
        return freqs
    

### Explicación/documentación

En este ejercicio se crean las clases 'WhooshBuilder' y 'WhooshIndex' las cuales implementan la funcionalidad de la creacion de un índice y la gestión del mismo respectivemente. Para afrontar este ejercicio nos hemos basado en el código de ejemplo proporcionado al principio de este documento, y para saber que métodos se requerían en cada una de las clases hemos ido siguiendo el código de prueba al final del documento e identificado cada uno de los métodos que se necesitaban. A posteriori, también hemos añadido algumna función que nos podía ser útil a parte de las explícitamente indicadas en el código de prueba (como es el caso de doc_terms_with_freq en Index). 


**WhooshBuilder**

El constructor de esta clase simplemente guardaba el index_path para su posterior uso. La mayor parte de la funcionalidad de esta clase reside en el método build. El método build crea un índice de búsqueda utilizando la biblioteca Whoosh. Toma una lista de colecciones como argumento y luego lee cada archivo en la lista y agrega sus contenidos al índice. El índice se crea en la ruta especificada en la variable self.index_path.

Si la lista de colecciones es un directorio, la función recorre cada archivo en el directorio y agrega sus contenidos al índice. Si la lista de colecciones es un archivo de texto (.txt), la función lee cada línea del archivo de texto y utiliza la biblioteca BeautifulSoup para extraer el contenido HTML de cada URL y agregarlo al índice. Si la lista de colecciones es un archivo ZIP (.zip), la función lee cada archivo dentro del archivo ZIP y agrega sus contenidos al índice.

Finalmente ncontramos el método commit el cual simplemente hace que los documentos agregados estén disponibles para la búsqueda. Esta función confirma los cambios y escribe los documentos agregados en el índice en el disco. 


**WhooshIndex**

La clase WhooshIndex se utiliza para realizar diferentes operaciones de búsqueda y análisis de texto sobre un índice Whoosh. Cada uno de los métodos es autodescriptivo por su nombre y simplemente sustituye operaciones básicas sobre un Indice Whoosh. 


Decidimos eliminar las clases abstractas Index y Builder ya que no tenian uso al estar definidas inmediatamente despues. 


## Ejercicio 1.2: Búsqueda (2pt)

Implementar la clase WhooshSearcher como subclase de Searcher.

In [2]:
import math
import re

def from_query_to_terms(text):
    return re.findall(r"[^\W\d_]+|\d+", text.lower())

In [3]:
class WhooshSearcher():
    """
    Whoosh Searcher.
    Class to search a Whoosh index.
    
    @param index_path : str
        Path to the index directory.
    @attribute index : whoosh.index.Index
        Whoosh index.
    @attribute searcher : whoosh.searching.Searcher
        Whoosh searcher.
    @attribute qparser : whoosh.qparser.QueryParser
        Whoosh query parser.
    """
    index = None
    searcher = None
    qparser = None

    def __init__(self, index_path):
        self.index = whoosh.index.open_dir(index_path)
        self.searcher = self.index.searcher()
        self.qparser = QueryParser("content", schema=self.index.schema)

    def search(self, query, cutoff):
        """
        search(query, cutoff)
            - Search the index for a query.
        @param query: query to search
        @param cutoff: number of results to return
        @return: list of tuples (path, score)
        """
        result = []
        search_results = list(self.searcher.search(self.qparser.parse(query)).items())[:cutoff]

        for docid, score in search_results:
            result.append((self.index.reader().stored_fields(docid)['path'], score))
        return result
        


### Explicación/documentación

La clase WhooshSearcher encapsula un buscador "Searcher" orientado a obtener resultados sobre el índice que obtiene como argumento.
Para cumplir su propósito hace uso de la librería whoosh, para abrir y leer el índice. Además del constructor, la clase WhooshSearcher
contiene un método search. Éste método devuelve los N documentos del índice que más puntuación obtienen, estando la puntuación directamente
relacionada con las apariciones y frecuencia de los términos introducidos como "query" en el argumento 1, siendo N el argumento 2.
Ya que utilizamos las funciones implementadas con la librería whoosh, no podemos ver de manera directa el código que se utiliza para obtener los
scores.

# Ejercicio 2: Modelo vectorial

Implementar dos modelos de ránking propios, basados en el modelo vectorial.

## Ejercicio 2.1: Producto escalar (2.5pt)

Implementar un modelo vectorial propio que utilice el producto escalar (sin dividir por las normas de los vectores) como función de ránking, por medio de la clase VSMDotProductSearcher, como subclase de Searcher.

Este modelo hará uso de la clase Index y se podrá probar con la implementación WhooshIndex (puedes ver un ejemplo de esto en la celda de prueba).

Además, la clase VSMDotProductSearcher será intercambiable con WhooshSearcher, como se puede ver en la celda de prueba, donde la función test_search utiliza una implementación u otra sin distinción.

In [4]:
from whoosh.searching import Searcher



def tf(freq):
    """
    tf(freq)
        - Return the term frequency.
    @param freq: term frequency
    """
    return 1 + math.log2(freq) if freq > 0 else 0


def idf(df, n):
    """
    idf(df, n)
        - Return the inverse document frequency.
    @param df: document frequency
    @param n: number of documents
    """
    return math.log2((n + 1) / (df + 0.5))



class VSMDotProductSearcher():
    """
    VSM Dot Product Searcher
    Class to search a Whoosh index using the VSM model with the dot product.

    @param index_path : str
        - Path to the index directory.
    """
    def __init__(self, index):
        self.index = index

    def search(self, query, cutoff):
        """
        search(query, cutoff)
            - Search the index for a query.
        @param query: query to search
        @param cutoff: number of results to return
        @return: list of tuples (path, score)
        """
        docs=[]
        terms = from_query_to_terms(query)
        for doc_id in range(self.index.ndocs()):
            score=0
            for term in terms:
                score += tf(self.index.term_freq(term, doc_id)) * idf(self.index.doc_freq(term), self.index.ndocs())
            if score > 0:
                docs.append([self.index.doc_path(doc_id), score])
        docs.sort(key=lambda x: x[1], reverse=True)
        return docs[0:cutoff]

### Explicación/documentación

La clase VSMDotProductSearcher es una subclase de Searcher que implementa un modelo vectorial de búsqueda utilizando el producto escalar sin dividir por las normas de los vectores. Esto se logra mediante el cálculo de la puntuación de cada documento en función de la frecuencia del término en el documento y la frecuencia del término en el índice. Para cada documento en el índice, se calcula su puntuación sumando las puntuaciones de todos los términos en la consulta que aparecen en el documento. Si la puntuación del documento es mayor que cero, se agrega a la lista de resultados.

La función tf calcula la frecuencia de término ajustada (tf) para un término en un documento. La función idf calcula el factor de ponderación inverso del documento (idf) para un término en el índice. La suma de los valores tf y idf para cada término se utiliza para calcular la puntuación de un documento.

La función search toma una consulta y un límite superior de resultados, y devuelve una lista de los documentos más relevantes ordenados por puntuación decreciente. La consulta se divide en términos y se busca la puntuación de cada documento en función de los términos en la consulta. Los resultados se devuelven como una lista de pares de documentos y puntuaciones.

Esta implementación es justificable porque el producto escalar es una medida común de similitud en espacios vectoriales, y no dividir por las normas de los vectores puede simplificar los cálculos y no afectar significativamente la calidad de los resultados. Además, la implementación utiliza las funciones tf e idf estándar en el procesamiento de lenguaje natural, lo que mejora la precisión de los resultados. En general, la implementación de VSMDotProductSearcher es una forma razonable y eficiente de realizar búsquedas en un índice de Whoosh.

### Ejercicio

Añadir a mano un documento a la colección docs1k.zip de manera que aparezca el primero para la consulta “obama family tree” para este buscador. Documentar cómo se ha conseguido y por qué resulta así.

### Solución

Para obtener el score con producto escalar multiplicamos la frecuencia de los términos en el documento por la función idf, siendo esta "log2((n + 1) / (df + 0.5))".
Para obtener un score lo más grande posible, por lo tanto, hay dos relaciones que podemos modificar.
Por un lado podemos aumentar la frecuencia de cada término (respecto del documento), que modifica directamente el valor del score.
Por otro lado, necesitamos que la función idf devuelva el valor más alto posible. Como el número de documentos es constante, lo más optimo es que (df + 0.5) sea
lo menor posible, siendo df la frecuencia total de los términos en búsqueda.
Por lo tanto, para obtener el mejor resultado posible hay que conseguir un balance entre la frecuencia de los términos por documento y la frecuencia de los términos en relación al resto de documentos del índice. Como la función idf escala de forma logarítmica, y en cambio, la cantidad de apariciones de cada término escala de forma lineal, hemos decidido utilizar un documento con la query utilizada ("obama family tree") repetida un gran número de veces.

## Ejercicio 2.2: Coseno (2pt)

Refinar la implementación del modelo para que calcule el coseno, definiendo para ello una clase VSMCosineSearcher. Para ello se necesitará extender Builder (o WhooshBuilder) con el cálculo de los módulos de los vectores, que deberán almacenarse en un fichero, en la carpeta de índice junto a los ficheros que genera cada índice. 

Pensad en qué parte del diseño interesa hacer esto, en concreto, qué clase y en qué momento tendría que calcular, devolver y/o almacenar estos módulos.

In [5]:
class VSMCosineSearcher():
    """
    VSMCosineSearcher
    Class to search a Whoosh index using the VSM with cosine similarity.
    
    @param index : WhooshSearcher
        - Whoosh index
    @attribute index : WhooshSearcher
        - Whoosh index
    @attribute modules : list
        - List of modules of the documents in the index
    """
    index = None
    modules = []

    def __init__(self, index):
        self.index = index
        self.modules = []
        for doc_id in range(index.ndocs()):
            div, divd, divq = 0, 0, 0
            for vec in index.doc_terms_with_freq(doc_id):
                divd = (tf(vec[1])) ** 2 
                divq = (idf(index.doc_freq(vec[0]), index.ndocs())) ** 2
                div += divd*divq
            div=math.sqrt(div)
            self.modules.append(div)




    def search(self, query, cutoff):
        """
        search(query, cutoff)
            - Search the index for a query.
        @param query: query to search
        @param cutoff: number of results to return
        @return: list of tuples (path, score)
        """
        docs=[]
        terms = from_query_to_terms(query)
        for doc_id in range(self.index.ndocs()):
            score=0
            for term in terms:
                score += ((tf(self.index.term_freq(term, doc_id)) * idf(self.index.doc_freq(term), self.index.ndocs())) / self.modules[doc_id])
            if score > 0:
                docs.append([self.index.doc_path(doc_id), score])
        docs.sort(key=lambda x: x[1], reverse=True)
        return docs[0:cutoff]


### Explicación/documentación

En esta nueva implementación, se ha refinado el modelo vectorial anterior para que calcule el coseno en lugar del producto escalar como función de ranking, lo que puede proporcionar resultados más precisos. Para ello, se ha creado la clase VSMCosineSearcher, que extiende la clase Searcher para permitir la búsqueda en el índice.

La parte más importante de esta nueva implementación es el cálculo de los módulos de los vectores. Los módulos se calculan en el constructor de VSMCosineSearcher y se almacenan en una lista llamada modules. Para calcular el módulo de un vector, se utiliza la fórmula:


cos = sum(tf(w) * idf(w))/ sqrt(sum(tf(w)^2 * idf(w)^2))
donde 𝑡𝑓 mide la “importancia” de los términos en los documentos e 𝑖𝑑𝑓 mide el poder de discriminación del término

Al inicializar el buscador guardamos la parte de abajo del algortimo, sqrt(sum(tf(w)^2 * idf(w)^2)). Esto nos ayuda a tener parte de la función ya realizada a la hora de satisfacer cada búsqueda.

Cada vez que se utiliza el método "search" de la clase, ejecuta el resto de la fórmula, sumando el score de los términos dependientes de la query

Finalmente, se ordenan los resultados por score en orden descendente y se devuelve un máximo de cutoff resultados.

En cuanto al almacenamiento de los módulos, se ha decidido que se almacenen en un fichero en la carpeta del índice, junto con los ficheros que genera cada índice. Para ello, se debe extender la clase Builder o WhooshBuilder para que calcule y almacene los módulos en el momento en que se crea el índice.

### Ejercicio

Añadir a mano un documento a la colección docs1k.zip de manera que aparezca el primero para la consulta “obama family tree” para este buscador. Documentar cómo se ha conseguido y por qué resulta así.

### Solución

Para obtener el mismo resultado para la búsqueda de "obama family tree" con este nuevo CosineSearcher, la implementación que debemos seguir es similar a la utilizada para el modelo de producto escalar, ya que a pesar de que la fórmula que utilizamos para obtener el score varien, las dependencias se mantienen, estando relacionado directamente con la frecuencia de los elementos de la query por documento. Por ello, igualmente, hemos utilizado un nuevo documento llenandolo de una gran repetición de los términos de la búsqueda para obtener el mayor score posible.

# Ejercicio 3: Estadísticas de frecuencias (1pt)

Utilizando las funcionalidades de la clase Index, implementar una función term_stats que calcule a) las frecuencias totales en la colección de los términos, ordenadas de mayor a menor, y b) el número de documentos que contiene cada término, igualmente de mayor a menor. Visualizar las estadísticas obtenidas en dos gráficas en escala log-log (dos gráficas por cada colección, seis gráficas en total), que se mostrarán en el cuaderno entregado.

De esta forma, podrás comprobar si las estadísticas de la colección siguen algún tipo de comportamiento esperado (como la conocida [Ley de Zipf](https://es.wikipedia.org/wiki/Ley_de_Zipf)).

In [6]:
import matplotlib.pyplot as plt

def get_file_name(path):
    dir_name, file_name = os.path.split(path)

    match = re.match(r'^(.*)\.(.*?)$', file_name)
    if match:
        return match.group(1)
    else:
        return file_name

def term_stats(index):
    # las frecuencias totales en la colección de los términos, ordenadas de mayor a menor,
    freqs = sorted(index.all_terms_with_freq(), key=lambda x: x[1], reverse=True)
    # Visualizar las estadísticas obtenidas en dos gráficas en escala log-log
    plt.loglog([x[1] for x in freqs])
    plt.xlabel("Term")
    plt.ylabel("Frequency")
    plt.title("Total term frequency on : "+get_file_name(index.path))
    plt.grid(True)
    plt.savefig("term_frequency_"+get_file_name(index.path)+".png")
    plt.clf()
    # el número de documentos que contiene cada término, igualmente de mayor a menor.
    docs_per_term = sorted([(term, index.doc_freq(term)) for term in index.all_terms()], key=lambda x: x[1], reverse=True)
    plt.loglog([x[1] for x in docs_per_term])
    plt.xlabel("Term")
    plt.ylabel("Documents")
    plt.title("Number of documents per term on : "+get_file_name(index.path))
    plt.grid(True)
    plt.savefig("documents_per_term_"+get_file_name(index.path)+".png")
    plt.clf()

### Explicación/documentación
Para obtener los diferentes plots, en el código al final del documento hemos añadido una linea justo después de la creación del Indice que llamaba a esta funcion. Esta función simplemente genera dos tipos de gráficos para cada una de las colecciones. Una gráfica muestra el numero de documentos en los que aparece cada término ordenado de mayor numero de apariciones a menor (documents_per_term_XXX.png) y la otra gráfica muestra  la frecuencia de cada uno de los términos enla colección. 

Es importante destacar que cuanto mayor numero de datos se recogen mas se parece la grafica resultante a la ley de Zipf la cual sigue la siguiente función P~1/nª.


### Imágenes de los plots:
![Alt text](plots/documents_per_term_toy.png)
![Alt text](plots/term_frequency_toy.png)
![Alt text](plots/documents_per_term_urls.png)
![Alt text](plots/term_frequency_urls.png)
![Alt text](plots/documents_per_term_docs.png)
![Alt text](plots/term_frequency_docs.png)


# Celda de prueba

Descarga los ficheros del curso de Moodle y coloca sus contenidos en una carpeta *collections* en el mismo directorio que este *notebook*. El fichero *toy.zip* hay que descomprimirlo para indexar la carpeta que contiene.

In [9]:
import os
import shutil
import time

def clear (index_path: str):
    if os.path.exists(index_path): shutil.rmtree(index_path)
    else: print("Creating " + index_path)
    os.makedirs(index_path)

def test_collection(collection_paths: list, index_path: str, word: str, query: str):
    start_time = time.time()
    print("=================================================================")
    print("Testing indices and search on " + str(len(collection_paths)) + " collections")

    # Let's create the folder if it did not exist
    # and delete the index if it did
    clear(index_path)

    # We now test building an index
    test_build(WhooshBuilder(index_path), collection_paths)

    # We now inspect the index
    index = WhooshIndex(index_path)
    test_read(index, word)

    print("------------------------------")
    print("Checking search results")
    test_search(WhooshSearcher(index_path), query, 5)
    test_search(VSMDotProductSearcher(WhooshIndex(index_path)), query, 5)
    test_search(VSMCosineSearcher(WhooshIndex(index_path)), query, 5)

def test_build(builder, collections: list):
    stamp = time.time()
    print("Building index with", type(builder))
    for collection in collections:
        print("Collection:", collection)
        # This function should index the received collection and add it to the index
        builder.build(collection)
    # When we commit, the information in the index becomes persistent
    # we can also save any extra information we may need
    # (and that cannot be computed until the entire collection is scanned/indexed)
    builder.commit()
    print("Done (", time.time() - stamp, "seconds )")
    print()

def test_read(index, word):
    stamp = time.time()
    print("Reading index with", type(index))
    print("Collection size:", index.ndocs())
    print("Vocabulary size:", len(index.all_terms()))
    terms = index.all_terms_with_freq()
    terms.sort(key=lambda tup: tup[1], reverse=True)
    print("  Top 5 most frequent terms:")
    for term in terms[0:5]:
        print("\t" + term[0] + "\t" + str(term[1]) + "=" + str(index.total_freq(term)))
    print()
    # More tests
    doc_id = 0
    print()
    print("  Frequency of word \"" + word + "\" in document " + str(doc_id) + " - " + index.doc_path(doc_id) + ": " + str(index.term_freq(word, doc_id)))
    print("  Total frequency of word \"" + word + "\" in the collection: " + str(index.total_freq(word)) + " occurrences over " + str(index.doc_freq(word)) + " documents")
    print("  Docs containing the word'" + word + "':", index.doc_freq(word))
    print("Done (", time.time() - stamp, "seconds )")
    print()


def test_search (engine, query, cutoff):
    stamp = time.time()
    print("  " + engine.__class__.__name__ + " for query '" + query + "'")
    for path, score in engine.search(query, cutoff):
        print(score, "\t", path)
    print()
    print("Done (", time.time() - stamp, "seconds )")
    print()


index_root_dir = "./index/"
collections_root_dir = "./collections/"
test_collection ([collections_root_dir + "toy/"], index_root_dir + "toy", "cc", "aa dd")
test_collection ([collections_root_dir + "urls.txt"], index_root_dir + "urls", "wikipedia", "information probability")
test_collection ([collections_root_dir + "docs1k.zip"], index_root_dir + "docs", "seat", "obama family tree")
test_collection ([collections_root_dir + "toy/", collections_root_dir + "urls.txt", collections_root_dir + "docs1k.zip"], index_root_dir + "all_together", "seat", "obama family tree")

Testing indices and search on 1 collections
Building index with <class '__main__.WhooshBuilder'>
Collection: ./collections/toy/
Done ( 0.026472091674804688 seconds )

Reading index with <class '__main__.WhooshIndex'>
Collection size: 4
Vocabulary size: 39
  Top 5 most frequent terms:
	aa	9.0=9.0
	bb	5.0=5.0
	sleep	5.0=5.0
	cc	3.0=3.0
	die	2.0=2.0


  Frequency of word "cc" in document 0 - ./collections/toy/d1.txt: 2
  Total frequency of word "cc" in the collection: 3.0 occurrences over 2 documents
  Docs containing the word'cc': 2
Done ( 0.0013790130615234375 seconds )

------------------------------
Checking search results
  WhooshSearcher for query 'aa dd'

Done ( 0.0009815692901611328 seconds )

  VSMDotProductSearcher for query 'aa dd'
4.0 	 ./collections/toy/d1.txt
1.7369655941662063 	 ./collections/toy/d2.txt
1.0 	 ./collections/toy/d3.txt

Done ( 0.0009028911590576172 seconds )

  VSMCosineSearcher for query 'aa dd'
1.0 	 ./collections/toy/d2.txt
0.7427813527082074 	 ./collectio

### Salida obtenida por el estudiante